In [2]:
import os
import json
import requests
import pandas as pd

# === 基础配置 ===
excel_path = "people_data.xlsx"
output_dir = "images/people"
undergrad_json = "undergrads.json"
grad_json = "grads.json"

# 创建图片保存目录
os.makedirs(output_dir, exist_ok=True)

# === 读取 Excel 数据 ===
df = pd.read_excel(excel_path, dtype=str).fillna("")

# 列名（请确保与Excel一致）
name_col = "1、姓名："
email_col = "2、个人邮箱"
research_col = "3、在校生填研究方向"
grad_col = "4、是否毕业，如毕业请填毕业时间"
photo_col = "6、个人照片（不超过 300*300,100kb, jpg格式）："

undergrads = []
grads = []

# === 遍历每一行数据 ===
for _, row in df.iterrows():
    name = row[name_col].strip()
    email = row[email_col].strip()
    research = row[research_col].strip()
    grad_status = row[grad_col].strip()
    photo_url = row[photo_col].strip()

    # 照片保存路径（初始）
    photo_filename = f"{name}.jpg"
    photo_path = os.path.join(output_dir, photo_filename)
    photo_web_path = f"/images/people/{photo_filename}"

    # === 下载照片 ===
    if photo_url:
        try:
            response = requests.get(photo_url, timeout=10)
            if response.status_code == 200:
                # 保存文件
                with open(photo_path, "wb") as f:
                    f.write(response.content)

                # 检查文件大小是否超出300KB
                size_kb = os.path.getsize(photo_path) / 1024
                if size_kb > 300:
                    new_name = f"OM{name}.jpg"
                    new_path = os.path.join(output_dir, new_name)
                    os.rename(photo_path, new_path)
                    photo_path = new_path
                    photo_web_path = f"/images/people/{new_name}"

            else:
                print(f"❌ 下载失败: {name} ({response.status_code})")

        except Exception as e:
            print(f"⚠️ 下载出错 {name}: {e}")

    # === 生成 JSON 条目 ===
    record = {
        "name": name,
        "photo": photo_web_path,
        "research": research,
        "email": email
    }

    # 判断是否毕业
    if grad_status in ["否", "未毕业", "在读", "no", "未完成"]:
        undergrads.append(record)
    else:
        grads.append(record)

# === 写入 JSON 文件 ===
with open(undergrad_json, "w", encoding="utf-8") as f:
    json.dump(undergrads, f, ensure_ascii=False, indent=4)

with open(grad_json, "w", encoding="utf-8") as f:
    json.dump(grads, f, ensure_ascii=False, indent=4)

print("✅ 数据处理完成！")
print(f"📄 未毕业学生: {undergrad_json} ({len(undergrads)}条)")
print(f"🎓 已毕业学生: {grad_json} ({len(grads)}条)")
print(f"🖼️ 照片保存路径: {output_dir}")


✅ 数据处理完成！
📄 未毕业学生: undergrads.json (35条)
🎓 已毕业学生: grads.json (18条)
🖼️ 照片保存路径: images/people


In [5]:
import os
from PIL import Image
from io import BytesIO

def process_om_images(image_dir):
    for filename in os.listdir(image_dir):
        if filename.startswith("OM") and filename.lower().endswith((".jpg", ".jpeg", ".png")):
            image_path = os.path.join(image_dir, filename)
            print(f"🖼️ 处理图片：{image_path}")

            try:
                # 使用 with 语句确保文件在操作完成后被关闭
                with Image.open(image_path) as img:
                    # 再次压缩到 300KB 以下
                    new_size = compress_image_to_limit(image_path, max_size_kb=300)
                    print(f"✅ 压缩完成: {filename}, 新大小 {new_size:.1f} KB")

                # 文件句柄已关闭，现在可以安全地重命名
                if new_size and new_size <= 300:
                    new_filename = filename.replace("OM", "", 1)
                    new_path = os.path.join(image_dir, new_filename)

                    # 确保不覆盖已有文件
                    if os.path.exists(new_path):
                        os.remove(new_path)

                    os.rename(image_path, new_path)
                    print(f"🔄 重命名为: {new_filename}")

            except Exception as e:
                print(f"⚠️ 处理图片 {filename} 失败: {e}")
# 你的图片目录
image_dir = "images/people"

# 遍历目录中所有以 "OM" 开头的文件
for filename in os.listdir(image_dir):
    if filename.startswith("OM") and filename.lower().endswith((".jpg", ".jpeg", ".png")):
        image_path = os.path.join(image_dir, filename)
        print(f"🖼️ 处理图片：{image_path}")

        try:
            # 打开图片
            img = Image.open(image_path)

            # === 示例操作1：再次压缩至300KB以下 ===
            # 这里直接复用前面定义的函数
            new_size = compress_image_to_limit(image_path, max_size_kb=300)
            print(f"✅ 压缩完成: {filename}, 新大小 {new_size:.1f} KB")

            # === 示例操作2：去掉“OM”前缀（若压缩成功） ===
            if new_size and new_size <= 300:
                new_filename = filename.replace("OM", "", 1)
                new_path = os.path.join(image_dir, new_filename)
                os.rename(image_path, new_path)
                print(f"🔄 重命名为: {new_filename}")

        except Exception as e:
            print(f"⚠️ 处理图片 {filename} 失败: {e}")


🖼️ 处理图片：images/people\OM任泽槟.jpg
✅ 压缩完成: OM任泽槟.jpg, 新大小 95.5 KB
⚠️ 处理图片 OM任泽槟.jpg 失败: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'images/people\\OM任泽槟.jpg' -> 'images/people\\任泽槟.jpg'
🖼️ 处理图片：images/people\OM党浩天.jpg
✅ 压缩完成: OM党浩天.jpg, 新大小 254.7 KB
⚠️ 处理图片 OM党浩天.jpg 失败: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'images/people\\OM党浩天.jpg' -> 'images/people\\党浩天.jpg'
🖼️ 处理图片：images/people\OM冷小雨.jpg
✅ 压缩完成: OM冷小雨.jpg, 新大小 104.8 KB
⚠️ 处理图片 OM冷小雨.jpg 失败: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'images/people\\OM冷小雨.jpg' -> 'images/people\\冷小雨.jpg'
🖼️ 处理图片：images/people\OM吴佳毅.jpg
✅ 压缩完成: OM吴佳毅.jpg, 新大小 263.5 KB
⚠️ 处理图片 OM吴佳毅.jpg 失败: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'images/people\\OM吴佳毅.jpg' -> 'images/people\\吴佳毅.jpg'
🖼️ 处理图片：images/people\OM周俊杰.jpg
✅ 压缩完成: OM周俊杰.jpg, 新大小 289.5 KB
⚠️ 处理图片 OM周俊杰.jpg 失败: [WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'images/people\\OM周俊杰.jpg' -> 'images/people\\周俊杰.jpg'
🖼️ 处理图片：images/people\OM孙赫男.jpg
✅ 压缩完成: OM孙赫男.jpg, 新大小 299.3 KB
⚠️ 处理图片 OM孙赫男.jpg 失败: [WinError 32] 另一个程序正在使用此文件，进程无

In [6]:
import os
import time

image_dir = "images/people"

for filename in os.listdir(image_dir):
    # 判断文件是否以 "OM" 开头
    if filename.startswith("OM"):
        old_path = os.path.join(image_dir, filename)
        new_filename = filename.replace("OM", "", 1)  # 只替换第一个OM
        new_path = os.path.join(image_dir, new_filename)

        # 如果目标文件已存在，先删除或重命名（避免冲突）
        if os.path.exists(new_path):
            print(f"⚠️ 目标文件已存在，跳过：{new_filename}")
            continue

        try:
            # 稍微延时，避免系统占用（Windows）
            time.sleep(0.05)
            os.rename(old_path, new_path)
            print(f"🔄 重命名：{filename} → {new_filename}")
        except Exception as e:
            print(f"❌ 重命名失败：{filename}，错误：{e}")


🔄 重命名：OM任泽槟.jpg → 任泽槟.jpg
🔄 重命名：OM党浩天.jpg → 党浩天.jpg
🔄 重命名：OM冷小雨.jpg → 冷小雨.jpg
🔄 重命名：OM吴佳毅.jpg → 吴佳毅.jpg
🔄 重命名：OM周俊杰.jpg → 周俊杰.jpg
🔄 重命名：OM孙赫男.jpg → 孙赫男.jpg
🔄 重命名：OM尹梓琦.jpg → 尹梓琦.jpg
🔄 重命名：OM崔冬航.jpg → 崔冬航.jpg
🔄 重命名：OM张亚龙.jpg → 张亚龙.jpg
🔄 重命名：OM李坚辉.jpg → 李坚辉.jpg
🔄 重命名：OM李洵楷.jpg → 李洵楷.jpg
🔄 重命名：OM杨一春.jpg → 杨一春.jpg
🔄 重命名：OM胡仲则.jpg → 胡仲则.jpg
🔄 重命名：OM胡旭阳.jpg → 胡旭阳.jpg
🔄 重命名：OM苏道涵.jpg → 苏道涵.jpg
🔄 重命名：OM范博文.jpg → 范博文.jpg
🔄 重命名：OM邬征宇.jpg → 邬征宇.jpg
🔄 重命名：OM邹知秋.jpg → 邹知秋.jpg
🔄 重命名：OM陈久健.jpg → 陈久健.jpg
🔄 重命名：OM陈子健.jpg → 陈子健.jpg
🔄 重命名：OM顾扬.jpg → 顾扬.jpg
🔄 重命名：OM马泽航.jpg → 马泽航.jpg
❌ 重命名失败：OM高森.jpg，错误：[WinError 32] 另一个程序正在使用此文件，进程无法访问。: 'images/people\\OM高森.jpg' -> 'images/people\\高森.jpg'
